<a href="https://colab.research.google.com/github/thinkdeepai/spark-training/blob/main/Spark_Training_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download and install dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar xf spark-3.2.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import os

# set the environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
# For this you need to create a free account on ngrok.com and enter your auth token below (replace AUHT_TOKEN)
NGROK_AUTH_TOKEN = "2CXOq3cZSdyxDgL97aR8zRo69KF_2PiUKLMyzpcxqq5TuLYQf"

# Export Spark history URL
from pyngrok import ngrok, conf

conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(4050, "http").public_url

print(f"Link to Spark History: {connection_string}")

Link to Spark History: https://fb4f-34-23-60-174.ngrok-free.app


In [7]:
# Please download the data here:
# https://raw.githubusercontent.com/apache/spark/master/examples/src/main/resources/people.json
!wget https://raw.githubusercontent.com/apache/spark/master/examples/src/main/resources/people.json

--2023-06-29 08:33:23--  https://raw.githubusercontent.com/apache/spark/master/examples/src/main/resources/people.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73 [text/plain]
Saving to: ‘people.json’

people.json         100%[===================>]      73  --.-KB/s    in 0s      

2023-06-29 08:33:23 (2.48 MB/s) - ‘people.json’ saved [73/73]



In [ ]:
df = spark.read.json("people.json")
# Displays the content of the DataFrame to stdout
df.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+


+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  25|  Kelly|
|  36|Stephen|
|  16| Andrew|
+----+-------+



In [ ]:
# $example on:untyped_ops$
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()
# root
# |-- age: long (nullable = true)
# |-- name: string (nullable = true)


root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
# Select only the "name" column
df.select("name").show()
# +-------+
# |   name|
# +-------+
# |Michael|
# |   Andy|
# | Justin|
# +-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|  Kelly|
|Stephen|
| Andrew|
+-------+



In [ ]:
#  *** TODO Select everybody, but increment the age by 1
df.select(df['name'], df['age'] + 1).show()
# +-------+---------+
# |   name|(age + 1)|
# +-------+---------+
# |Michael|     null|
# |   Andy|       31|
# | Justin|       20|
# +-------+---------+


In [ ]:
# *** TODO Select people older than 21
df.filter(df['age'] > 21).show()
# +---+----+
# |age|name|
# +---+----+
# | 30|Andy|
# +---+----+



+---+-------+
|age|   name|
+---+-------+
| 30|   Andy|
| 25|  Kelly|
| 36|Stephen|
+---+-------+



In [ ]:
# *** TODO Count people by age
df.groupBy("age").count().show()
# +----+-----+
# | age|count|
# +----+-----+
# |  19|    1|
# |null|    1|
# |  30|    1|
# +----+-----+
# $example off:untyped_ops$



+----+-----+
| age|count|
+----+-----+
|  19|    1|
|  25|    1|
|null|    1|
|  36|    1|
|  30|    1|
|  16|    1|
+----+-----+



In [ ]:
# $example on:run_sql$
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+
# $example off:run_sql$



+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  25|  Kelly|
|  36|Stephen|
|  16| Andrew|
+----+-------+



In [ ]:
# $example on:global_temp_view$
# Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()

# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+


In [ ]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+
# $example off:global_temp_view$

In [ ]:
# spark.sql("SELECT name FROM global_temp.people WHERE age >= 13 AND age <= 19").show()
df.select("name").where("age >= 13 AND age <= 19").show()
df.select("name").where("age <= 19").show()

+------+
|  name|
+------+
|Justin|
|Andrew|
+------+

+------+
|  name|
+------+
|Justin|
|Andrew|
+------+



In [ ]:
# teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
# teenagerNamesDF = spark.sql("SELECT avg(age) FROM people")
from pyspark.sql.functions import avg
# df.agg({"age" : "avg"}).show()
df.select(avg("age")).show()
# teenagerNamesDF.show()
# +------+
# |  name|
# +------+
# |Justin|
# +------+
# $example off:basic_parquet_example$

+--------+
|avg(age)|
+--------+
|    25.2|
+--------+



In [ ]:
# Alternatively, a DataFrame can be created for a JSON dataset represented by
# an RDD[String] storing one JSON object per string
jsonStrings = ['{"name":"Yin","address":{"city":"Columbus","state":"Ohio"}}']
otherPeopleRDD = sc.parallelize(jsonStrings)
otherPeople = spark.read.json(otherPeopleRDD)

# otherPeople.show()
# +---------------+----+
# |        address|name|
# +---------------+----+
# |[Columbus,Ohio]| Yin|
# +---------------+----+
# $example off:json_dataset$

In [ ]:
# Using Parquet

df.write.parquet("people.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("people.parquet")

# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")

# ***** TODO
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")

teenagers.show()
# +------+
# |  name|
# +------+
# |Justin|
# +------+
# $example off:basic_parquet_example$

In [ ]:
# Inferring the Schema Using Reflection
from pyspark.sql import Row

# Load a text file and convert each line to a Row.
lines = sc.textFile("people.txt")

parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as a :class:`pyspark.RDD` of :class:`Row`.
teenagers.show()

teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()

for name in teenNames:
    print(name)
# Name: Justin

+------+
|  name|
+------+
|Justin|
|Andrew|
+------+

Name: Justin
Name: Andrew


In [ ]:
# ***** TODO
# Try adding : ".explain(extended=True)" to teenagers or any SparkSQL query
# Observe the streaming query plans generated for the query

# teenagers.explain(extended=True)
# df.explain(extended=True)
df.select(avg("age")).explain(extended=True)

== Parsed Logical Plan ==
'Project [avg('age) AS avg(age)#625]
+- Relation[age#140L,name#141] json

== Analyzed Logical Plan ==
avg(age): double
Aggregate [avg(age#140L) AS avg(age)#625]
+- Relation[age#140L,name#141] json

== Optimized Logical Plan ==
Aggregate [avg(age#140L) AS avg(age)#625]
+- Project [age#140L]
   +- Relation[age#140L,name#141] json

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[avg(age#140L)], output=[avg(age)#625])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#902]
   +- *(1) HashAggregate(keys=[], functions=[partial_avg(age#140L)], output=[sum#629, count#630L])
      +- FileScan json [age#140L] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex[file:/content/people.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint>

